In [2]:
import pandas as pd

In [3]:
def get_conf_matrix(seqs,seqs_obs,imputed,impute_values,seqs_time_since_start,imputed_time_since_start):
    def pad(list_,max_len,pad_value=None):
        for i in range(len(list_),max_len):
            list_.append(pad_value)
        return list_
    max_len = max(len(seqs),len(seqs_obs),len(imputed))
    imputed_padded = pad(imputed,max_len)
    seqs_obs_padded = pad(seqs_obs,max_len)
    seqs_padded = pad(seqs,max_len)
    seqs_time_since_start_pad = pad(seqs_time_since_start,max_len)
    imputed_time_since_start_pad = pad(imputed_time_since_start,max_len)
    confusion_df=pd.DataFrame()
    confusion_df['predicted']= imputed_padded
    confusion_df['observed']= seqs_obs_padded
    confusion_df['actual']= seqs_padded
    confusion_df['masked']=0
    confusion_df['predicted_mask']=0
#     confusion_df['true_mask']=0
    confusion_df['time_since_start']=seqs_time_since_start_pad
    confusion_df['imputed_time_since_start']=imputed_time_since_start

    for impute in impute_values:
        s="predicted=={} and observed!=predicted".format(impute)
        indices=(confusion_df.query(s).index)
        for index in indices:
            confusion_df.loc[index,"predicted_mask"]=1
        s="actual=={} and observed!=actual".format(impute)
        indices=(confusion_df.query(s).index)
        for index in indices:
            confusion_df.loc[index,"masked"]=1
            
#         display(confusion_df)
#         same value in consecutive samples not handled
       
#             indices=confusion_df.query("masked=maskes_args").index
#         when masked is 1 shift time by +1 and -1 and take values
#         when predicted mask is 1 note down corresponding time_since_start
#         check intervals
    
        confusion_df['time_since_start_args_1']=confusion_df['time_since_start'].shift(1)
        confusion_df['time_since_start_args_2']=confusion_df['time_since_start'].shift(-1)
        
        confusion_df['time_since_start_args_1'].fillna(0,inplace=True)
        confusion_df['time_since_start_args_2'].fillna(99999,inplace=True)
#         display(confusion_df)
        confusion_df['masked_args']=confusion_df['masked'].shift(1)
        indices=confusion_df.query("masked==1 and masked_args==1").index
        for index in indices:
            confusion_df.loc[index,"time_since_start_args_1"]=None
        confusion_df['masked_args_1']=confusion_df['masked'].shift(-1)
        indices=confusion_df.query("masked==1 and masked_args_1==1").index
        for index in indices:
            confusion_df.loc[index,"time_since_start_args_2"]=None

        confusion_df['time_since_start_args_2'].fillna(method='bfill',inplace=True)
        confusion_df['time_since_start_args_1'].fillna(method='ffill',inplace=True)
        confusion_df['imputed_time_args']=confusion_df['imputed_time_since_start']*confusion_df['predicted_mask']
        confusion_df['start_time']=confusion_df['time_since_start_args_1']*confusion_df['masked']
        confusion_df['end_time']=confusion_df['time_since_start_args_2']*confusion_df['masked']
#         indices=df.query("masked==1").index
#         for i in index:
#             df.loc[i,"start_time"]=df.loc[i,"time_since_start_args_2"]
#             df.loc[i,"end_time"]=df.loc[i,"time_since_start_args_1"]
            
    
#         indices=df.query("predicted_mask==1").index
#         for i in index:
#             df.loc[i,imputed_time_args]=df.loc[i,imputed_time_since_start]
        #look for intervals
    
        indices=confusion_df.query("masked==1").index
#         time intervals in seq 
        intervals=[]
#         predicted time in imputed data
        predicted_vals=[]
        
        for index in indices:
            intervals.append((confusion_df.loc[index,"start_time"],confusion_df.loc[index,"end_time"]))
        indices=confusion_df.query("predicted_mask==1").index
        for index in indices:
            predicted_vals.append(confusion_df.loc[index,"imputed_time_args"])
            

            

        unique_intervals = list(set(intervals))
        print(unique_intervals)
        print(predicted_vals)
        true_count = 0
        for p_times in predicted_vals:
            for interval in unique_intervals:
                if (p_times > interval[0]) & (p_times <interval[1]):
                    true_count +=1
                    break
        
                    
        print(true_count)
        sum_mask=confusion_df["masked"].sum()
        sum_pred_mask=confusion_df["predicted_mask"].sum()
        
        
#     display(confusion_df)
    
    return (true_count,sum_mask,sum_pred_mask),confusion_df

## Test Cases

### Case 5: Fractional missingness

In [4]:
actual = [8,6,5,5,5,5,5,4,3,2]
obs = [8,6,5,4,3,2]
imputed = [8,5,6,5,5,5,4,3,2]
(true_count,sum_mask,sum_pred_mask),df = get_conf_matrix(actual,obs,imputed,[5],[1,2,3,4,5,6,7,8,9,10],[1,2,3,4,5,6,7,8,9])

[(3.0, 8.0)]
[2.0, 4.0, 5.0, 6.0]
3


In [5]:
df

,predicted,observed,actual,masked,predicted_mask,time_since_start,imputed_time_since_start,time_since_start_args_1,time_since_start_args_2,masked_args,masked_args_1,imputed_time_args,start_time,end_time
0,8.0,8.0,8,0,0,1,1.0,0.0,2.0,NaN,0.0,0.0,0.0,0.0
1,5.0,6.0,6,0,1,2,2.0,1.0,3.0,0.0,0.0,2.0,0.0,0.0
2,6.0,5.0,5,0,0,3,3.0,2.0,4.0,0.0,1.0,0.0,0.0,0.0
3,5.0,4.0,5,1,1,4,4.0,3.0,8.0,0.0,1.0,4.0,3.0,8.0
4,5.0,3.0,5,1,1,5,5.0,3.0,8.0,1.0,1.0,5.0,3.0,8.0
5,5.0,2.0,5,1,1,6,6.0,3.0,8.0,1.0,1.0,6.0,3.0,8.0
6,4.0,NaN,5,1,0,7,7.0,3.0,8.0,1.0,0.0,0.0,3.0,8.0
7,3.0,NaN,4,0,0,8,8.0,7.0,9.0,1.0,0.0,0.0,0.0,0.0
8,2.0,NaN,3,0,0,9,9.0,8.0,10.0,0.0,0.0,0.0,0.0,0.0
9,NaN,NaN,2,0,0,10,NaN,9.0,99999.0,0.0,NaN,NaN,0.0,0.0


In [14]:
df[['actual','observed','predicted','masked','predicted_mask','time_since_start','imputed_time_since_start']]

,actual,observed,predicted,masked,predicted_mask,time_since_start,imputed_time_since_start
0,8,8.0,8.0,0,0,1,1.0
1,6,6.0,5.0,0,1,2,2.0
2,5,5.0,6.0,0,0,3,3.0
3,5,4.0,5.0,1,1,4,4.0
4,5,3.0,5.0,1,1,5,5.0
5,5,2.0,5.0,1,1,6,6.0
6,5,NaN,4.0,1,0,7,7.0
7,4,NaN,3.0,0,0,8,8.0
8,3,NaN,2.0,0,0,9,9.0
9,2,NaN,NaN,0,0,10,NaN


In [15]:
print("Precision:",true_count/sum_pred_mask)
print("Recall:",true_count/sum_mask)

Precision: 0.75
Recall: 0.75


### End event imputation

In [16]:
actual = [8,6,5,5,5,5,5,4,3,2,5]
obs = [8,6,5,5,5,5,5,4,3,2]
imputed = [8,6,5,5,5,5,5,4,3,2,5]
(true_count,sum_mask,sum_pred_mask),df = get_conf_matrix(actual,obs,imputed,[5],[1,2,3,4,5,6,7,8,9,10,11],[1,2,3,4,5,6,7,8,9,10,11])

[(10.0, 99999.0)]
[11]
1


In [17]:
df[['actual','observed','predicted','masked','predicted_mask','time_since_start','imputed_time_since_start']]

,actual,observed,predicted,masked,predicted_mask,time_since_start,imputed_time_since_start
0,8,8.0,8,0,0,1,1
1,6,6.0,6,0,0,2,2
2,5,5.0,5,0,0,3,3
3,5,5.0,5,0,0,4,4
4,5,5.0,5,0,0,5,5
5,5,5.0,5,0,0,6,6
6,5,5.0,5,0,0,7,7
7,4,4.0,4,0,0,8,8
8,3,3.0,3,0,0,9,9
9,2,2.0,2,0,0,10,10


In [18]:
print("Precision:",true_count/sum_pred_mask)
print("Recall:",true_count/sum_mask)

Precision: 1.0
Recall: 1.0


### Consecutive is imputed but the same is not present in actual

In [19]:
actual = [8,6,5,5,4,3,2]
obs = [8,6,5,4,3,2]
imputed = [8,6,5,5,5,5,5,4,3,2]
(true_count,sum_mask,sum_pred_mask),df = get_conf_matrix(actual,obs,imputed,[5],[1,2,3,4,5,6,7],[1,2,3,4,5,6,7,8,9,10])

[(3.0, 5.0)]
[4, 5, 6, 7]
1


In [20]:
df[['actual','observed','predicted','masked','predicted_mask','time_since_start','imputed_time_since_start']]

,actual,observed,predicted,masked,predicted_mask,time_since_start,imputed_time_since_start
0,8.0,8.0,8,0,0,1.0,1
1,6.0,6.0,6,0,0,2.0,2
2,5.0,5.0,5,0,0,3.0,3
3,5.0,4.0,5,1,1,4.0,4
4,4.0,3.0,5,0,1,5.0,5
5,3.0,2.0,5,0,1,6.0,6
6,2.0,NaN,5,0,1,7.0,7
7,NaN,NaN,4,0,0,NaN,8
8,NaN,NaN,3,0,0,NaN,9
9,NaN,NaN,2,0,0,NaN,10


In [21]:
print("Precision:",true_count/sum_pred_mask)
print("Recall:",true_count/sum_mask)

Precision: 0.25
Recall: 1.0
